In [2]:
import gc
import os
import warnings
import numpy as np
import pandas as pd
from scipy.stats.mstats import winsorize
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from pypfopt import black_litterman, risk_models
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier
from cvxopt import matrix, solvers
from numpy.linalg import inv, solve

gc.collect()
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'cvxopt'

In [ ]:
import os
os.getcwd()

'c:\\Users\\HYU\\Downloads\\FERM\\FERM'

In [ ]:
def create_datasets(top_n, freq="M"):
    """
    monthly_dict 폴더의 parquet 파일들을 순회하면서 
    cshtrm 상위 20개 기업을 추출하여 price_df, returns_df, cshthrm_df, conm_df를 생성

    Parameters:
    - monthly_dict_path: monthly_dict 폴더 경로
    - top_n: 추출할 상위 기업 수 (기본값: 20)

    Returns:
    - price_df: 가격 데이터 (인덱스: 날짜, 컬럼: tic)
    - returns_df: 수익률 데이터 (인덱스: 날짜, 컬럼: tic)
    - cshthrm_df: 시가총액 데이터 (인덱스: 날짜, 컬럼: tic)
    - conm_df: 회사명 데이터 (인덱스: 날짜, 컬럼: tic)
    """

    # 결과를 저장할 딕셔너리
    price_data = {}
    returns_data = {}
    cshthrm_data = {}
    conm_data = {}

    if freq == "M":
        monthly_dict_path = "data/monthly_dict"
        vol = 'cshtrm'
    elif freq == "D":
        monthly_dict_path = "data/daily_dict"
        vol = 'cshoc'
        
    '''base_dir = os.getcwd()

    if freq == "M":
        monthly_dict_path = os.path.join(base_dir, "data", "monthly_dict")
        vol = 'cshtrm'
    elif freq == "D":
        monthly_dict_path = os.path.join(base_dir, "data", "daily_dict")
        vol = 'cshoc'
        '''

    # monthly_dict 폴더의 모든 parquet 파일 찾기
    monthly_dict_dir = os.listdir(monthly_dict_path)

    for file_path in sorted(monthly_dict_dir):
        file_index = file_path.split('.')[0]
        df = pd.read_parquet(os.path.join(monthly_dict_path, file_path))
        df['cap'] = df['adj_closed']*df[vol]
        valid_data = df.dropna(subset=['cshtrd'])

        top_companies = valid_data.nlargest(top_n, 'cshtrd')

        # 각 데이터프레임에 데이터 추가
        i = 0
        for _, row in top_companies.iterrows():
            i += 1
            # 가격 데이터 (prccd)
            if 'adj_closed' in row and pd.notna(row['adj_closed']):
                price_data[(file_index, 'Asset_'+str(i))] = row['adj_closed']

            # 수익률 데이터 (prccd를 사용하여 계산)
            if 'adj_closed_pct_change' in row and pd.notna(row['adj_closed_pct_change']):
                returns_data[(file_index, 'Asset_'+str(i))
                             ] = row['adj_closed_pct_change']

            # 시가총액 데이터 (cshtrm)
            if vol in row and pd.notna(row['cap']):
                cshthrm_data[(file_index, 'Asset_'+str(i))] = row['cap']

            # 회사명 데이터 (conm)
            if 'conm' in row and pd.notna(row['conm']):
                conm_data[(file_index, 'Asset_'+str(i))] = row['conm']

    # price_df 생성
    if price_data:
        price_df = pd.DataFrame.from_dict(price_data, orient='index')
        price_df.index = pd.MultiIndex.from_tuples(
            price_df.index, names=['date', 'tic'])
        price_df = price_df.unstack(level=1)
        price_df.columns = price_df.columns.droplevel(0)
    else:
        price_df = pd.DataFrame()

    # returns_df 생성
    if returns_data:
        returns_df = pd.DataFrame.from_dict(returns_data, orient='index')
        returns_df.index = pd.MultiIndex.from_tuples(
            returns_df.index, names=['date', 'tic'])
        returns_df = returns_df.unstack(level=1)
        returns_df.columns = returns_df.columns.droplevel(0)
    else:
        returns_df = pd.DataFrame()

    # cshthrm_df 생성
    if cshthrm_data:
        cshthrm_df = pd.DataFrame.from_dict(cshthrm_data, orient='index')
        cshthrm_df.index = pd.MultiIndex.from_tuples(
            cshthrm_df.index, names=['date', 'tic'])
        cshthrm_df = cshthrm_df.unstack(level=1)
        cshthrm_df.columns = cshthrm_df.columns.droplevel(0)
    else:
        cshthrm_df = pd.DataFrame()

    # conm_df 생성
    if conm_data:
        conm_df = pd.DataFrame.from_dict(conm_data, orient='index')
        conm_df.index = pd.MultiIndex.from_tuples(
            conm_df.index, names=['date', 'tic'])
        conm_df = conm_df.unstack(level=1)
        conm_df.columns = conm_df.columns.droplevel(0)
    else:
        conm_df = pd.DataFrame()

    return price_df, returns_df, cshthrm_df, conm_df


In [ ]:
class DynamicSlidingWindowBL:
    """
    Dynamic Sliding Window Algorithm using PyPortfolioOpt Black-Litterman
    """

    def __init__(self, n_assets=10, initial_window=60, eta=0.95, h=0.1, c_minus=0.9, c_plus=1.1, risk_aversion=1.0):
        self.n_assets = n_assets
        self.M = initial_window
        self.eta = eta
        self.h = h
        self.c_minus = c_minus
        self.c_plus = c_plus
        self.risk_aversion = risk_aversion
        # Results storage
        self.expected_returns = []
        self.portfolio_weights = {}
        self.portfolio_volatilities = []
        self.window_sizes = []
        self.market_returns = []
        self.bl_returns = []
        self.views_history = []

    def generate_random_data(self, n_periods=1000, n_factors=5):
        """Generate random market data"""
        np.random.seed(42)

        # Generate factor data (AR(1) process)
        factors = np.zeros((n_periods, n_factors))
        for i in range(n_factors):
            factors[0, i] = np.random.normal(0, 0.02)
            for t in range(1, n_periods):
                factors[t, i] = 0.9 * factors[t-1, i] + \
                    np.random.normal(0, 0.02)

        # Generate asset returns (factor model + noise)
        beta = np.random.normal(0, 0.5, (self.n_assets, n_factors))
        alpha = np.random.normal(0.001, 0.002, self.n_assets)

        returns = np.zeros((n_periods, self.n_assets))
        for t in range(n_periods):
            returns[t] = alpha + factors[t] @ beta.T + \
                np.random.normal(0, 0.02, self.n_assets)

        # Generate price data for PyPortfolioOpt
        prices = np.zeros((n_periods, self.n_assets))
        prices[0] = 100  # Initial price
        for t in range(1, n_periods):
            prices[t] = prices[t-1] * (1 + returns[t])

        # Convert to DataFrame
        price_df = pd.DataFrame(prices,
                                columns=[f'Asset_{i+1}' for i in range(self.n_assets)])

        # Generate time-varying market caps
        # Initial market caps (random distribution)
        initial_caps = np.random.dirichlet(np.ones(self.n_assets)) * 1000000

        # Market cap evolution (follows price changes with some noise)
        cap_df = np.zeros((n_periods, self.n_assets))
        cap_df[0] = initial_caps

        for t in range(1, n_periods):
            # Market cap changes follow price changes with some additional noise
            price_change = prices[t] / prices[t-1]
            cap_change = price_change * \
                (1 + np.random.normal(0, 0.01, self.n_assets))  # Add some noise
            cap_df[t] = cap_df[t-1] * cap_change

        # Convert to DataFrame
        cap_df = pd.DataFrame(cap_df,
                              columns=[f'Asset_{i+1}' for i in range(self.n_assets)])

        return returns, factors, price_df, cap_df

    def calculate_views_from_factors(self, factors, returns):
        """Calculate views using factor model"""
        alphas = []
        betas = []

        for i in range(self.n_assets):
            # Elastic Net regression
            model = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=1000)

            scaler_X = StandardScaler()
            scaler_y = StandardScaler()

            X_scaled = scaler_X.fit_transform(factors)
            y_scaled = scaler_y.fit_transform(
                returns[:, i].reshape(-1, 1)).ravel()

            model.fit(X_scaled, y_scaled)

            beta = model.coef_ / scaler_X.scale_ * scaler_y.scale_
            alpha = scaler_y.mean_[0] - np.sum(beta * scaler_X.mean_)

            alphas.append(alpha)
            betas.append(beta)

        alphas = np.array(alphas)
        betas = np.array(betas)

        # Views setup
        P = np.eye(self.n_assets)  # Direct views for each asset
        # Factor model based expected returns
        Q = alphas + np.mean(factors @ betas.T, axis=0)

        # Views uncertainty
        Omega = np.diag(np.var(returns, axis=0)) * 0.025

        return P, Q, Omega

    def calculate_portfolio_volatility(self, weights, returns):
        """Calculate portfolio volatility"""
        portfolio_returns = returns @ weights
        volatility = np.std(portfolio_returns)
        return volatility

    def adjust_window_size(self, current_vol, previous_vol):
        """Dynamically adjust window size based on volatility"""
        if current_vol >= (1 + self.h) * previous_vol:
            # Increase volatility -> decrease window size
            self.M = int(self.c_minus * self.M)
        elif current_vol <= (1 - self.h) * previous_vol:
            # Decrease volatility -> increase window size
            self.M = int(self.c_plus * self.M)

        # Limit window size
        self.M = max(15, min(self.M, 200))

    def dict2numpy(self, w_bl, mu_bl):
        # Convert to numpy arrays
        # Handle OrderedDict or dict for weights
        if isinstance(w_bl, dict):
            w_bl = np.array(list(w_bl.values()))
        elif hasattr(w_bl, 'values'):
            w_bl = w_bl.values
            if hasattr(w_bl, '__iter__') and not isinstance(w_bl, np.ndarray):
                w_bl = np.array(list(w_bl))
            else:
                w_bl = np.array(w_bl)
        else:
            w_bl = np.array(w_bl)

        # Handle OrderedDict or dict for returns
        if isinstance(mu_bl, dict):
            mu_bl = np.array(list(mu_bl.values()))
        elif hasattr(mu_bl, 'values'):
            mu_bl = mu_bl.values
            if hasattr(mu_bl, '__iter__') and not isinstance(mu_bl, np.ndarray):
                mu_bl = np.array(list(mu_bl))
            else:
                mu_bl = np.array(mu_bl)
        else:
            mu_bl = np.array(mu_bl)

        return w_bl, mu_bl
    
    #-------------------------------#
    # *** 직접 구현 새롭게 추가/변경 사항
    def Pi(self, market_caps, risk_aversion, cov_matrix): # risk_free_rate은 0으로 고정
        """
        market weight기반으로 implied equilibrium excess return(pi)를 구함
        Π = δΣwmkt

        Parameters:
        - market_caps: 각 자산의 시가총액. {ticker: cap} dict or pd.Series
        - risk_aversion: 위험 회피 계수. positive float
        - cov_matrix: covariance matrix of asset returns. pd.DataFrame

        Returns:
        - implied equilibrium excess return. pd.Series
        """
        if not isinstance(cov_matrix, pd.DataFrame):
            warnings.warn(
                "If cov_matrix is not a dataframe, market cap index must be aligned to cov_matrix",
                RuntimeWarning,
            )
        market_caps_series = pd.Series(market_caps)
        mkt_w = market_caps_series / market_caps_series.sum()
        
        return risk_aversion * cov_matrix.dot(mkt_w)
    
    def posterior_dist(self, cov_matrix, pi, P=None, Q=None, Omega=None, tau = 0.025):
        """
        return: μ_BL, Σ_BL
        μ_BL = M⁻¹ * b,  where
            M = (τΣ)⁻¹ + Pᵀ Ω⁻¹ P
            b = (τΣ)⁻¹ π + Pᵀ Ω⁻¹ q
        Σ_BL = Σ + (M)⁻¹
        """
        if P is None or Q is None:
            # 전망이 없으면 prior distribution만
            return pi.copy(), cov_matrix.copy()
        
        # (τΣ)⁻¹ 계산
        I = np.eye(cov_matrix.shape[0])
        inv_ts = solve(tau * cov_matrix, I)

        # Ω⁻¹ 계산
        I_omega = np.eye(Omega.shape[0])
        inv_omega = solve(Omega, I_omega)

        P_inv_om = P.T @ inv_omega # Pᵀ Ω⁻¹

        M = inv_ts + P_inv_om @ P
        b = inv_ts @ pi + P_inv_om @ Q
        inv_M = solve(M, I)

        mu_bl = solve(M,b)
        sigma_bl = cov_matrix + inv_M
        return mu_bl, sigma_bl

    def mean_var_opt(self, mu, cov_matrix, risk_aversion, long_only = True):
        '''
        cvxopt 양식:
            min (1/2) wᵀ (δΣ) w + (-μᵀ) w
            s.t. 1ᵀ w = 1
                (-I)w ≤ 0 (no shortselling)
        long_only: 롱온리 제약 여부
        return: (BL 사후 분포 만족하는) 가중치
        '''
        n = len(mu) # 자산 개수
        # 공분산 singular 방지
        eps = 1e-6

        if isinstance(cov_matrix, pd.DataFrame):
            cov_matrix = cov_matrix.values
        if isinstance(mu, (pd.Series, pd.DataFrame)):
            mu = np.array(mu).ravel()

        P = matrix(risk_aversion * (cov_matrix + eps * np.eye(n))) # n*n
        q = matrix(-mu, (n,1)) # n*1

        # 부등식 제약. G = -I, h = [0,..,0]ᵀ
        G, h = (matrix(-np.eye(n)), matrix(np.zeros(n))) if long_only else (None, None)
        
        # 만약 가중치 상한선 둘 거면 코드 변경    
        '''G_list, h_list = [], []
        if long_only:
            G_list.append(-np.eye(n))
            h_list.append(np.zeros(n))
        if w_max is not None:
            G_list.append(np.eye(n))
            h_list.append(np.full(n, w_max))

        G, h = (matrix(np.vstack(G_list), tc='d'), matrix(np.concatenate(h_list), tc='d')) if len(G_list) > 0 else (None, None)'''

        # 등식 제약
        A = matrix(np.ones(n),(1,n)) # A = [1,1,...,1]
        b = matrix(1.0)

        # Optimization
        solvers.options['show_progress'] = False
        sol = solvers.qp(P,q,G,h,A,b)
        w = np.array(sol['x']).ravel()
        return w
    
    # 여기까지
    #--------------------------------------------------------------------------------#

    def run_algorithm(self, returns_df, factor_df, cap_df, n_iterations=50):
        """Run the complete algorithm"""
        n_periods = len(returns_df)
        returns = returns_df.values
        factors = factor_df.values
        t = self.M

        print(f"Dynamic Sliding Window Algorithm Started")
        print(f"Initial window size: {self.M}")

        for iteration in range(n_iterations):
            if t + self.M >= n_periods:
                break

            # Step 1: Data Collection
            r_data = returns[t-self.M:t]
            f_data = factors[t-self.M:t]
            r_df = returns_df.iloc[t-self.M:t]
            date = str(returns_df.iloc[t].name)
            # Step 2: Calculate covariance matrix using PyPortfolioOpt

            cov_matrix = r_df.cov()
            # Step 3: Calculate market implied returns using PyPortfolioOpt
            # Get current market caps from cap_df
            # Use market cap from previous period
            current_market_caps = cap_df.iloc[t-1][r_df.columns]

            # print(current_market_caps)
            # *** package black_litterman.market_implied_prior_returns() -> Pi로 변경
            pi = self.Pi(
                current_market_caps, risk_aversion=self.risk_aversion, cov_matrix=cov_matrix
            )

            # Step 4: Calculate views using factor model
            P, Q, Omega = self.calculate_views_from_factors(f_data, r_data)
            # print(pi.values)

            # *** Step 5: Optain posterior distribution by Black Litterman
            mu_bl, sigma_bl = self.posterior_dist(cov_matrix, pi, P=P, Q=Q, Omega=Omega, tau = 0.025)

            # *** Step 6: 논문의 공분산 구현
            rrT = np.outer(r_data[-1], r_data[-1])
            S_bl = self.eta * sigma_bl + (1 - self.eta) * rrT

            # *** NaN 값 & inf 처리
            mu_mean = np.nanmean(mu_bl)
            mu_bl = np.nan_to_num(mu_bl, nan=mu_mean, posinf=0.0, neginf=0.0)
            S_bl = np.nan_to_num(S_bl, nan=0.0, posinf=0.0, neginf=0.0)

            # *** 2) 최적화: 공매도 금지 (0 ≤ w ≤ 1)
            w_bl = self.mean_var_opt(mu_bl, sigma_bl, risk_aversion= self.risk_aversion, long_only=True)

            '''# Step 5: Apply Black-Litterman model using PyPortfolioOpt
            bl = BlackLittermanModel(
                cov_matrix, pi=pi, P=P, Q=Q, omega=Omega, tau=0.025)

            # Step 6: Get Black-Litterman returns and weights
            mu_bl = bl.bl_returns()     # posterior expected returns (Series)
            rrT = np.outer(r_data[-1], r_data[-1])
            S_bl = self.eta * bl.bl_cov() + (1 - self.eta) * rrT
            # cov_matrix = risk_models.exp_cov(price_data, returns_data=True)

            # NaN값 처리
            mu_bl = mu_bl.fillna(mu_bl.mean())
            S_bl = S_bl.fillna(0)
            # inf값 처리
            mu_bl = mu_bl.replace([np.inf, -np.inf], 0)
            S_bl = S_bl.replace([np.inf, -np.inf], 0)'''


            '''# 2) 최적화: 공매도 금지 (0 ≤ w ≤ 1)
            # short-sale not allowed
            ef = EfficientFrontier(mu_bl, S_bl, weight_bounds=(0, 1))
            # ef.max_quadratic_utility(self.risk_aversion)
            ef.max_sharpe(risk_free_rate=0.0)
            w_bl = ef.clean_weights()

            w_bl, mu_bl = self.dict2numpy(w_bl, mu_bl)'''

            # Step 7: Execute transactions and evaluate volatility
            current_returns = returns[t:t+self.M]
            current_vol = self.calculate_portfolio_volatility(
                w_bl, current_returns)

            # Step 8: Dynamic window size adjustment
            if iteration > 0:
                previous_vol = self.portfolio_volatilities[-1]
                self.adjust_window_size(current_vol, previous_vol)

            # Store results
            self.expected_returns.append(mu_bl.copy())
            self.portfolio_weights[date] = w_bl.copy()
            self.portfolio_volatilities.append(current_vol)
            self.window_sizes.append(self.M)
            self.market_returns.append(pi.copy())
            self.bl_returns.append(mu_bl.copy())
            self.views_history.append(Q.copy())

            # Step 9: Update time
            t += self.M

            if iteration % 10 == 0:
                print(
                    f"Iteration {iteration}: Window size = {self.M}, Volatility = {current_vol:.4f}")

        print(f"Algorithm completed: {len(self.portfolio_weights)} iterations")

    def get_results_summary(self):
        """Get algorithm results summary"""
        print(f"\n=== Algorithm Results Summary ===")
        print(f"Final window size: {self.M}")
        print(
            f"Average portfolio volatility: {np.mean(self.portfolio_volatilities):.4f}")
        print(f"Volatility std: {np.std(self.portfolio_volatilities):.4f}")


In [ ]:
# dsa = DynamicSlidingWindowBL(
#     n_assets=20,           # 10 assets
#     initial_window=60,     # Initial window size 60
#     eta=0.95,              # EWMA decay factor
#     h=0.1,                 # Volatility threshold for window adjustment
#     risk_aversion=3.0,     # Risk aversion parameter
#     c_minus=0.9,           # Window decrease factor
#     c_plus=1.1             # Window increase factor
# )
# sim_returns, sim_factors, sim_price_df, sim_cap_df = dsa.generate_random_data(
#     n_periods=1000, n_factors=5)

# print(f"sim_returns.shape: {sim_returns.shape}")
# print(f"sim_factors.shape: {sim_factors.shape}")
# print(f"sim_price_df.shape: {sim_price_df.shape}")
# print(f"sim_cap_df.shape: {sim_cap_df.shape}")

# dsa.run_algorithm(pd.DataFrame(sim_returns, index=sim_price_df.index, columns=sim_price_df.columns), pd.DataFrame(sim_factors),
#                   sim_cap_df, n_iterations=30)

# # Display results
# dsa.get_results_summary()

In [3]:
n_assets = 20
freq = 'D'

price_df, returns_df, cshthrm_df, conm_df = create_datasets(n_assets, freq)

price_df = price_df.loc[:'2025-07-31', :].dropna()
returns_df = returns_df.loc[:'2025-07-31', :].dropna()
cshthrm_df = cshthrm_df.loc[:'2025-07-31', :].dropna()
conm_df = conm_df.loc[:'2025-07-31', :].dropna()

factor_df = pd.read_csv('data/factors.csv', index_col=0)
factor_df.index = factor_df.index.astype(str)
factor_df.index = pd.to_datetime(factor_df.index)
factor_df = factor_df.resample(freq).last().loc[price_df.index, :].ffill()
factor_df = factor_df.iloc[:, :-1]

NameError: name 'create_datasets' is not defined

In [ ]:
# 데이터 shape과 인덱스 확인
print(f"returns_df.shape: {returns_df.shape}")
print(f"factor_df.shape: {factor_df.shape}")
print(f"cshthrm_df.shape: {cshthrm_df.shape}")
print(f"returns_df.index range: {returns_df.index[0]} to {returns_df.index[-1]}")
print(f"cshthrm_df.index range: {cshthrm_df.index[0]} to {cshthrm_df.index[-1]}")
print(f"cshthrm_df.index length: {len(cshthrm_df.index)}")
print(f"returns_df.index length: {len(returns_df.index)}")


returns_df.shape: (3990, 20)
factor_df.shape: (3990, 5)
cshthrm_df.shape: (3990, 20)
returns_df.index range: 2010-01-05 to 2025-07-31
cshthrm_df.index range: 2010-01-05 to 2025-07-31
cshthrm_df.index length: 3990
returns_df.index length: 3990


In [ ]:
dsa = DynamicSlidingWindowBL(
    n_assets=n_assets,           # 10 assets
    initial_window=60,     # Initial window size 60
    eta=0.2,              # EWMA decay factor
    h=0.1,                 # Volatility threshold for window adjustment
    c_minus=0.9,           # Window decrease factor
    c_plus=1.1,             # Window increase factor
    risk_aversion=2.0
)

dsa.run_algorithm(returns_df, factor_df, cshthrm_df, n_iterations=140)

# Display results
dsa.get_results_summary()


Dynamic Sliding Window Algorithm Started
Initial window size: 60
Iteration 0: Window size = 60, Volatility = 0.1549
Iteration 10: Window size = 80, Volatility = 0.1224
Iteration 20: Window size = 59, Volatility = 0.2297
Iteration 30: Window size = 52, Volatility = 1.3890
Iteration 40: Window size = 38, Volatility = 0.5866
Iteration 50: Window size = 34, Volatility = 14.8970
Iteration 60: Window size = 36, Volatility = 2.3921
Iteration 70: Window size = 39, Volatility = 3.2649
Iteration 80: Window size = 30, Volatility = 2.0568
Algorithm completed: 83 iterations

=== Algorithm Results Summary ===
Final window size: 29
Average portfolio volatility: 2.8843
Volatility std: 5.6020


In [ ]:
weight_df = pd.DataFrame(dsa.portfolio_weights, index=price_df.columns).T
equal_weight_df = pd.DataFrame(1 / len(weight_df.columns), index=weight_df.index, columns=weight_df.columns)
print(weight_df.shape)
returns_df_winsorized = returns_df.apply(
    lambda x: winsorize(x, limits=[0.01, 0.01]), axis=0
)

weight_df.to_csv('data/weight_df.csv')
returns_df.to_csv('data/returns_df.csv')

(83, 20)
